# Analyser du texte

**IMPORTANT** : Créez une ressource *Analytique de texte* ou *Text analytics* et renseignez `SERVEUR` avec votre point de terminaison (endpoint). Récupérez également la clé et renseignez là lors de l'exécution de la première cellule.

In [8]:
from getpass import getpass

CLE = getpass("Clé")
SERVEUR = 'https://textanalysis00.cognitiveservices.azure.com/'  # Votre point de terminaison

TEXTE1 = """
    Passepartout, les mains dans les poches, se rendit donc vers le port Victoria, 
    regardant les palanquins, les brouettes à voile, encore en faveur dans 
    le Céleste Empire, et toute cette foule de Chinois, de Japonais et 
    d’Européens, qui se pressait dans les rues.
"""
# Jules Verne, Le tour du monde en qutre-vingt jours, Chapitre XIX

TEXTE2 = """
    SATURDAY morning was come, and all the summer world was bright and fresh,
    and brimming with life. There was a song in every heart; and if the heart 
    was young the music issued at the lips. There was cheer in every face and 
    a spring in every step. The locust-trees were in bloom and the fragrance 
    of the blossoms filled the air.
"""
# Mark Twain, The Adventures of Tom Sawyer, Chapter II

TEXTE3 = """
    Deja á la luna verme con luz tranquila y suave;
    Deja que el alba envíe su resplandor fugaz,
    Deja gemir al viento con su murmullo grave,
    Y si desciende y posa sobre mi cruz un ave
    Deja que el ave entone su cantico de paz.
"""
# Jose Rizal, Mi Ultimo Adiós

TEXTE4 = """
    Hélas ! Combien de temps faudra-t-il vous redire
    À vous tous, que c'était à vous de les conduire,
    Qu'il fallait leur donner leur part de la cité,
    Que votre aveuglement produit leur cécité ;
    D'une tutelle avare on recueille les suites,
    Et le mal qu'ils vous font, c'est vous qui le leur fîtes.
    Vous ne les avez pas guidés, pris par la main,
    Et renseignés sur l'ombre et sur le vrai chemin ;
    Vous les avez laissés en proie au labyrinthe.
    Ils sont votre épouvante et vous êtes leur crainte ;
"""
# Victor Hugo, L'année terrible, Poésie XII

## Sans SDK

### Évaluation du sentiment

In [9]:
import requests
from pprint import pprint

reponse = requests.post(
    url=SERVEUR + '/text/analytics/v2.1/sentiment',
    headers={'Ocp-Apim-Subscription-Key': CLE},
    json={'documents': [
        {'id': '1', 'language': 'fr', 'text': TEXTE1},
        {'id': '2', 'language': 'en', 'text': TEXTE2},
        {'id': '3', 'language': 'es', 'text': TEXTE3},
        {'id': '4', 'language': 'fr', 'text': TEXTE4}
    ]}
)

In [10]:
pprint(sorted(reponse.json()['documents'], key=lambda doc: -doc['score']))

[{'id': '2', 'score': 0.9599582552909851},
 {'id': '1', 'score': 0.7140833735466003},
 {'id': '3', 'score': 0.6403309106826782},
 {'id': '4', 'score': 0.3863636255264282}]


### Mots-clés

In [11]:
import requests
from pprint import pprint

reponse = requests.post(
    url=SERVEUR + '/text/analytics/v2.1/keyPhrases',
    headers={'Ocp-Apim-Subscription-Key': CLE},
    json={'documents': [
        {'id': '1', 'language': 'fr', 'text': TEXTE1},
        {'id': '2', 'language': 'en', 'text': TEXTE2},
        {'id': '3', 'language': 'es', 'text': TEXTE3},
        {'id': '4', 'language': 'fr', 'text': TEXTE4}
    ]}
)

In [15]:
[ f"{doc['id']} : {', '.join(doc['keyPhrases'])}" 
  for doc in sorted(reponse.json()['documents'], key=lambda doc: doc['id'])
]

['1 : brouettes, voile, foule de Chinois, Passepartout, mains, poches, Céleste Empire, Japonais, palanquins, faveur',
 '2 : heart, bloom, summer world, locust-trees, music, lips, fragrance, blossoms, air, spring, step, song, morning, life',
 '3 : luna, luz tranquila, alba, resplandor fugaz, viento, murmullo grave, posa, cruz, cantico de paz',
 '4 : ombre, main, tutelle avare, suites, proie, labyrinthe, vrai chemin, épouvante, crainte, part de la cité, aveuglement, cécité, temps, mal']

## Avec SDK

In [3]:
%pip install azure-cognitiveservices-language-textanalytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 8.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials
from pprint import pprint

In [16]:
apiTexte = TextAnalyticsClient(SERVEUR, CognitiveServicesCredentials(CLE))

reponse = apiTexte.sentiment(documents=[
    {'id': '1', 'language': 'fr', 'text': TEXTE1},
    {'id': '2', 'language': 'en', 'text': TEXTE2},
    {'id': '3', 'language': 'es', 'text': TEXTE3},
    {'id': '4', 'language': 'fr', 'text': TEXTE4}
])

In [18]:
pprint([doc.__dict__ for doc in sorted(reponse.documents, key=lambda doc: -doc.score)])

[{'additional_properties': {},
  'id': '2',
  'score': 0.9599582552909851,
  'statistics': None},
 {'additional_properties': {},
  'id': '1',
  'score': 0.7140833735466003,
  'statistics': None},
 {'additional_properties': {},
  'id': '3',
  'score': 0.6403309106826782,
  'statistics': None},
 {'additional_properties': {},
  'id': '4',
  'score': 0.3863636255264282,
  'statistics': None}]


In [20]:
reponse = apiTexte.key_phrases(documents=[
    {'id': '1', 'language': 'fr', 'text': TEXTE1},
    {'id': '2', 'language': 'en', 'text': TEXTE2},
    {'id': '3', 'language': 'es', 'text': TEXTE3},
    {'id': '4', 'language': 'fr', 'text': TEXTE4}
], show_stats=True)

In [24]:
[ f"{doc.id} : {', '.join(doc.key_phrases)}"
  for doc in sorted(reponse.documents, key=lambda doc: doc.id)
]

['1 : brouettes, voile, foule de Chinois, Passepartout, mains, poches, Céleste Empire, Japonais, palanquins, faveur',
 '2 : heart, bloom, summer world, locust-trees, music, lips, fragrance, blossoms, air, spring, step, song, morning, life',
 '3 : luna, luz tranquila, alba, resplandor fugaz, viento, murmullo grave, posa, cruz, cantico de paz',
 '4 : ombre, main, tutelle avare, suites, proie, labyrinthe, vrai chemin, épouvante, crainte, part de la cité, aveuglement, cécité, temps, mal']